In [2]:
import importlib
import os
os.environ['R_HOME'] = '/fs/home/liuzhaoyang/biosoft/mambaforge/envs/MS/lib/R'
import sys
sys.path.append('/fs/home/liuzhaoyang/project/MetaSpace/scripts/')
import preprocessing as MSpre
import processing as MSpp
import model as MS


# 
import pandas as pd
import numpy as np
import scanpy as sc
from pathlib import Path
import logging

import torch

import anndata2ri
from rpy2.robjects.packages import importr
from rpy2.robjects import r, pandas2ri

In [4]:
# os.environ['R_HOME'] = '/fs/home/liuzhaoyang/biosoft/mambaforge/envs/MetaSpace/lib/R'
# %load_ext rpy2.ipython

In [109]:
importlib.reload(MSpre)
# importlib.reload(MS)

<module 'preprocessing' from '/fs/home/liuzhaoyang/project/MetaSpace/scripts/preprocessing.py'>

## tested

- load_ST_data

In [84]:
tnp_obj = sc.read_10x_h5('/mnt/Venus/home/liuzhaoyang/data/10x_online_ST_datasets/CytAssist_11mm_FFPE_Human_Colorectal_Cancer/CytAssist_11mm_FFPE_Human_Colorectal_Cancer_filtered_feature_bc_matrix.h5')

/fs/home/liuzhaoyang/biosoft/anaconda3/envs/MetaSpace/lib/python3.11/site-packages/anndata/_core/anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [85]:
tnp_obj

AnnData object with n_obs × n_vars = 9080 × 18085
    var: 'gene_ids', 'feature_types', 'genome'

In [86]:
tnp_obj.var_names

Index(['SAMD11', 'NOC2L', 'KLHL17', 'PLEKHN1', 'PERM1', 'HES4', 'ISG15',
       'AGRN', 'RNF223', 'C1orf159',
       ...
       'MT-ND2', 'MT-CO2', 'MT-ATP6', 'MT-CO3', 'MT-ND3', 'MT-ND4L', 'MT-ND4',
       'MT-ND5', 'MT-ND6', 'MT-CYB'],
      dtype='object', length=18085)

In [30]:
tnp_obj.var_names_make_unique()

In [31]:
tnp_obj

AnnData object with n_obs × n_vars = 9080 × 18085
    var: 'gene_ids', 'feature_types', 'genome'

In [23]:
tmp_df = pd.read_csv('/mnt/Venus/home/liuzhaoyang/data/10x_online_ST_datasets/CytAssist_11mm_FFPE_Human_Colorectal_Cancer/ST_sct_counts.tsv',
                    sep='\t')

In [ ]:
tnp_obj = SCTransform(tnp_obj,
                        min_cells=5,
                        gmean_eps=1,
                        n_genes=2000,
                        n_cells=None, #use all cells
                        bin_size=500,
                        bw_adjust=3,
                    inplace=False)



In [77]:
?SCTransform

Object `SCTransform` not found.


In [83]:
tmp_obj.var_names

Index(['AACAATCCGAGTGGAC-1', 'AACAATGGAACCACAT-1', 'AACAATTATCTGCGCC-1',
       'AACACCAGCCTACTCG-1', 'AACACCATTCGCATAC-1', 'AACACCGAATGTCTCA-1',
       'AACACCGAGCTTGGAT-1', 'AACACCTAAGCATTGC-1', 'AACACGCAGATAACAA-1',
       'AACACGGCACCGATAG-1',
       ...
       'TGTTGCCGGAATGATC-1', 'TGTTGCCGTTCGACCA-1', 'TGTTGCTCGATGTCTG-1',
       'TGTTGGATAGATCAGA-1', 'TGTTGGCCTGTAGCGG-1', 'TGTTGGTGAGCGGACG-1',
       'TGTTGGTGCGCACGAG-1', 'TGTTGGTGCGCTTCGC-1', 'TGTTGGTGCGGAATCA-1',
       'TGTTGGTGGACTCAGG-1'],
      dtype='object', length=9080)

In [7]:
tmp_obj = MSpre.prepare_ST('/mnt/Venus/home/liuzhaoyang/data/10x_online_ST_datasets/CytAssist_11mm_FFPE_Human_Colorectal_Cancer/ST_sct_counts.tsv',
                           '',
                          sample_prefix='G1')

In [111]:
tmp_obj

AnnData object with n_obs × n_vars = 9080 × 18066

In [ ]:
# -------------------------- #
# SCTransform v2 Seurat ver. #
# -------------------------- #

anndata2ri.activate()
pandas2ri.activate()

seurat = importr('Seurat')
mat = tmp_obj.X

# Set names for the input matrix
cell_names = tmp_obj.obs_names
gene_names = tmp_obj.var_names
r.assign('mat', mat.T)
r.assign('cell_names', cell_names)
r.assign('gene_names', gene_names)
r('colnames(mat) <- cell_names')
r('rownames(mat) <- gene_names')

seurat = importr('Seurat')
r('seurat_obj <- CreateSeuratObject(mat)')

# Run
r('seurat_obj <- SCTransform(seurat_obj, verbose=F, vst.flavor="v2")')

filter_genes = r('rownames(mat)[!rownames(mat) %in% rownames(seurat_obj)]')
tmp_obj = tmp_obj[:,~np.isin(tmp_obj.var_names, filter_genes)]

# ----------- #
# write files #
# ----------- #

# ST raw counts
tmp_out = pd.DataFrame(tmp_obj.X).T
tmp_out.index = tmp_obj.var_names
tmp_out.columns = tmp_obj.obs_names
tmp_out.to_csv(Path(save_dir,'ST_counts.tsv'), sep='\t')

# ST SCT normalized
tmp_out = pd.DataFrame(r['as.matrix'](r('seurat_obj@assays$SCT@data')))
tmp_out.index = r('rownames(seurat_obj@assays$SCT@data)')
tmp_out.columns = r('colnames(seurat_obj@assays$SCT@data)')
tmp_out.to_csv(Path(save_dir,'ST_sct_norm.tsv'), sep='\t')
tmp_obj.layers['SCT_data'] = tmp_out.T

# ST SCT updated counts
tmp_out = pd.DataFrame(r['as.matrix'](r('seurat_obj@assays$SCT@counts')))
tmp_out.index = r('rownames(seurat_obj@assays$SCT@counts)')
tmp_out.columns = r('colnames(seurat_obj@assays$SCT@counts)')
tmp_out.to_csv(Path(save_dir,'ST_sct_counts.tsv'), sep='\t')
tmp_obj.layers['SCT_counts'] = tmp_out.T

# save scanpy obj
tmp_obj.write_h5ad(Path('ST_sct_obj.h5ad'))